In [2]:
import gym
import numpy as np
import operator
from IPython.display import clear_output
from time import sleep
import random
import itertools
import tqdm

tqdm.monitor_interval = 0

In [3]:
def create_random_policy(env):
     policy = {}
     for key in range(0, env.observation_space.n):
#           print(env.observation_space)
          current_end = 0
          p = {}
          for action in range(0, env.action_space.n):
               p[action] = 1 / env.action_space.n
          policy[key] = p
     print(policy)
     return policy

In [4]:
def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
         Q[key] = {a: 0.0 for a in range(0, env.action_space.n)}
    return Q

In [5]:
def run_game(env, policy, display=True):
     env.reset()
     episode = []
     finished = False

     while not finished:
          s = env.env.s
          if display:
               clear_output(True)
               env.render()
               sleep(1)

          timestep = []
          timestep.append(s)
          n = random.uniform(0, sum(policy[s].values()))
#           print("sum is ", sum(policy[s].values()))
          top_range = 0
#           print("n is ", n)
#           print("s is ", s)
#           print("items are ", policy[s].items())
          for prob in policy[s].items():
#                 print("probs are ", prob)
#                 input()
                top_range += prob[1]
                if n < top_range:
                      action = prob[0]
                      break 
          state, reward, finished, info = env.step(action)
          timestep.append(action)
          timestep.append(reward)

          episode.append(timestep)

     if display:
          clear_output(True)
          env.render()
          sleep(1)
     return episode

In [6]:
def test_policy(policy, env):
      wins = 0
      r = 100
      for i in range(r):
            w = run_game(env, policy, display=False)[-1][-1]
            if w == 1:
                  wins += 1
      return wins / r

In [18]:
def monte_carlo_e_soft(env, episodes=100, policy=None, epsilon=0.01):
    if not policy:
        policy = create_random_policy(env)  # Create an empty dictionary to store state action values    
    Q = create_state_action_dictionary(env, policy) # Empty dictionary for storing rewards for each state-action pair
    returns = {} # 3.
    
    for _ in range(episodes): # Looping through episodes
        print(policy)
        print()
        print()
        G = 0 # Store cumulative reward in G (initialized at 0)
        episode = run_game(env=env, policy=policy, display=False) # Store state, action and value respectively 
        
        # for loop through reversed indices of episode array. 
        # The logic behind it being reversed is that the eventual reward would be at the end. 
        # So we have to go back from the last timestep to the first one propagating result from the future.
        
        for i in reversed(range(0, len(episode))):   
            s_t, a_t, r_t = episode[i] 
            state_action = (s_t, a_t)
            G += r_t # Increment total reward by reward on current timestep
            
            if not state_action in [(x[0], x[1]) for x in episode[0:i]]: # 
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]   
                    
                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                print(Q[s_t][a_t])
                Q_list = list(map(lambda x: x[1], Q[s_t].items())) # Finding the action with maximum value
                print("Q_list", Q_list)
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                print("indices is ", indices)
                max_Q = random.choice(indices)
                print("max_Q", max_Q)
                A_star = max_Q # 14.
                
                for a in policy[s_t].items(): # Update action probability for s_t in policy
                    if a[0] == A_star:
                        policy[s_t][a[0]] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                    else:
                        policy[s_t][a[0]] = (epsilon / abs(sum(policy[s_t].values())))

    return policy

In [19]:
env = gym.make('FrozenLake8x8-v0')
policy = create_random_policy(env)
print("policy is ")
print(policy)
episode = run_game(env=env, policy=policy, display=False)
print("trajectory is ")
print(episode)
print()


{0: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 1: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 2: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 3: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 4: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 5: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 6: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 7: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 8: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 9: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 10: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 11: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 12: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 13: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 14: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 15: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 16: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 17: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 18: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 19: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 20: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 21: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 22: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 23: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 2

In [21]:
policy = monte_carlo_e_soft(env, episodes=100)
test_policy(policy, env)

{0: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 1: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 2: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 3: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 4: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 5: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 6: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 7: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 8: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 9: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 10: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 11: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 12: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 13: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 14: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 15: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 16: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 17: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 18: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 19: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 20: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 21: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 22: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 23: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 2

{0: {0: 0.009376577444384577, 1: 0.9993820617941619, 2: 0.004873085452040325, 3: 0.004907141280860681}, 1: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 2: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 3: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 4: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 5: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 6: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 7: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 8: {0: 1.0, 1: 0.005714285714285714, 2: 0.006641366223908918, 3: 0.007921697807306687}, 9: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 10: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 11: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 12: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 13: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 14: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 15: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 16: {0: 0.009731014768710808, 1: 0.009733562527980511, 2: 1.2845617759952808, 3: 0.00762252165110296}, 17: {0: 0.01, 1: 1.003157894736842, 2: 0.006608695652173913, 3: 0.007875463156528886}, 18: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.2

indices is  [0, 1, 2, 3]
max_Q 0
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 0
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 0
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 0
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 0
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 1
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 3
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 3
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 2
0.0
Q_list [0.0, 0.0, 0.0, 0.0]
indices is  [0, 1, 2, 3]
max_Q 2
{0: {0: 0.007369130338024453, 1: 0.0073824526635166855, 2: 0.9973957271132299, 3: 0.009808442031864883}, 1: {0: 0.009861518936292881, 1: 0.4077054560628179, 2: 0.023398992308202155, 3: 1.01242735849548}, 2: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 3: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 4: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 5: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 6: {0: 0.25, 1

{0: {0: 0.009808458095715837, 1: 0.3383918785176646, 2: 0.027549796456465662, 3: 1.0159363952070741}, 1: {0: 0.006880449925333477, 1: 0.006894591488160062, 2: 0.009527426386960122, 3: 0.999655027543006}, 2: {0: 0.01, 1: 1.003157894736842, 2: 0.006608695652173913, 3: 0.007875463156528886}, 3: {0: 0.01, 1: 0.013157894736842105, 2: 0.01911468812877264, 3: 1.024214635878446}, 4: {0: 0.009801256494123296, 1: 0.33246221318680363, 2: 1.0180248217417238, 3: 0.00730881961008639}, 5: {0: 0.009731014768710808, 1: 0.009733562527980511, 2: 1.2845617759952808, 3: 0.00762252165110296}, 6: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 7: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 8: {0: 0.009834329692132722, 1: 0.009811926847319753, 2: 0.009789829698991405, 3: 1.2863173737693414}, 9: {0: 0.997857729407619, 1: 0.004423270489776357, 2: 0.009823551525122551, 3: 0.009804395131418425}, 10: {0: 0.009801256494123296, 1: 1.3224622131868036, 2: 0.007424861145583358, 3: 0.007420544361886223}, 11: {0: 0.25, 1: 0.25, 2: 0.25,

{0: {0: 0.009770824857387974, 1: 0.009746205454647567, 2: 0.00972207073743784, 3: 0.9997195277106035}, 1: {0: 0.9997755772312343, 1: 0.004960680105104792, 2: 0.004965443711336409, 3: 0.004976717104390806}, 2: {0: 0.009731014768710808, 1: 0.9997335625279805, 2: 0.009766113915072707, 3: 0.009736091987639153}, 3: {0: 0.009376577444384577, 1: 0.9993820617941619, 2: 0.004873085452040325, 3: 0.004907141280860681}, 4: {0: 0.009801256494123296, 1: 0.33246221318680363, 2: 1.0180248217417238, 3: 0.00730881961008639}, 5: {0: 0.009731014768710808, 1: 0.009733562527980511, 2: 1.2845617759952808, 3: 0.00762252165110296}, 6: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 7: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}, 8: {0: 0.00760020802068864, 1: 0.007613134963559546, 2: 0.0076259004917236025, 3: 0.9976385054838142}, 9: {0: 0.997857729407619, 1: 0.004423270489776357, 2: 0.009823551525122551, 3: 0.009804395131418425}, 10: {0: 0.007423304963825939, 1: 0.007436431946397719, 2: 0.3366420486859444, 3: 1.01786118100027}

...

{0: {0: 0.9997834804234992, 1: 0.004964230665366603, 2: 0.004970528456449325, 3: 0.004976858148252546}, 1: {0: 0.007184762295671483, 1: 0.007198335752773324, 2: 0.9994531731158699, 3: 0.004926686205052623}, 2: {0: 0.00749331223685196, 1: 0.9975062372756197, 2: 0.004918470369564696, 3: 0.00982899275181815}, 3: {0: 0.9997716512086301, 1: 0.004962059380243233, 2: 0.004969203481096893, 3: 0.009808794623160397}, 4: {0: 0.9976416672316314, 1: 0.004354461625781327, 2: 0.00436473959277419, 3: 0.009862032192266904}, 5: {0: 0.9998367848131946, 1: 0.009814212142624035, 2: 0.009761461547859325, 3: 0.009715602263061835}, 6: {0: 0.009890773345421615, 1: 0.42654948276016863, 2: 1.0124372021863575, 3: 0.006878475896675452}, 7: {0: 0.009718486702657828, 1: 0.00971860503040927, 2: 0.25680997365529284, 3: 1.0249670941874818}, 8: {0: 0.00976418747453172, 1: 0.0097637306164324, 2: 0.29201310580666173, 3: 1.0211230592415592}, 9: {0: 0.00986264826804592, 1: 0.009840735079827223, 2: 0.35202245460710313, 3: 1.

{0: {0: 0.009855177208138974, 1: 1.3937664223684296, 2: 0.007074292162432629, 3: 0.007063779394493829}, 1: {0: 0.007184762295671483, 1: 0.007198335752773324, 2: 0.9994531731158699, 3: 0.004926686205052623}, 2: {0: 0.009806353807471807, 1: 0.999784160880628, 2: 0.00976240285592526, 3: 0.009716455273806213}, 3: {0: 0.9998086171200707, 1: 0.00980826148773963, 2: 0.009761860613844866, 3: 0.009716402182449905}, 4: {0: 0.9976416672316314, 1: 0.004354461625781327, 2: 0.00436473959277419, 3: 0.009862032192266904}, 5: {0: 0.9998367848131946, 1: 0.009814212142624035, 2: 0.009761461547859325, 3: 0.009715602263061835}, 6: {0: 0.009890773345421615, 1: 0.42654948276016863, 2: 1.0124372021863575, 3: 0.006878475896675452}, 7: {0: 0.009718486702657828, 1: 0.00971860503040927, 2: 0.25680997365529284, 3: 1.0249670941874818}, 8: {0: 0.009783481145944168, 1: 0.9997617090731976, 2: 0.004958703980715839, 3: 0.004965032657061512}, 9: {0: 0.009895339390021513, 1: 1.419754022582945, 2: 0.006950169724716383, 3: 

{0: {0: 0.007053381615803069, 1: 0.007067348203687999, 2: 1.343872050933667, 3: 0.007325703778104511}, 1: {0: 0.009815826073831574, 1: 0.9997905409596632, 2: 0.0049652772542051245, 3: 0.009808745832047334}, 2: {0: 0.009717517850733107, 1: 0.00971835680245714, 2: 1.2469720844746142, 3: 0.007836226536492616}, 3: {0: 0.009717274523671278, 1: 0.25638528311822734, 2: 1.0250163571372057, 3: 0.007687368162295633}, 4: {0: 0.9976416672316314, 1: 0.004354461625781327, 2: 0.00436473959277419, 3: 0.009862032192266904}, 5: {0: 0.9998367848131946, 1: 0.009814212142624035, 2: 0.009761461547859325, 3: 0.009715602263061835}, 6: {0: 0.009890773345421615, 1: 0.42654948276016863, 2: 1.0124372021863575, 3: 0.006878475896675452}, 7: {0: 0.009718486702657828, 1: 0.00971860503040927, 2: 0.25680997365529284, 3: 1.0249670941874818}, 8: {0: 0.9973772720677129, 1: 0.004267881533136882, 2: 0.004277998104876797, 3: 0.00986861276524444}, 9: {0: 0.006927431177159711, 1: 0.9969417033051917, 2: 0.009825534780062373, 3:

{0: {0: 0.9998074365775762, 1: 0.004970096821634557, 2: 0.004976127723608311, 3: 0.004982158274658698}, 1: {0: 0.9973413302218369, 1: 0.004255976845935888, 2: 0.004266105285169362, 3: 0.009869545186571285}, 2: {0: 0.009889837897156176, 1: 0.41819113099824873, 2: 1.0128350163261108, 3: 0.006916281828462763}, 3: {0: 0.007705186476083958, 1: 0.007717136260247617, 2: 0.9995396659509745, 3: 0.009778469888636347}, 4: {0: 0.009840360804384574, 1: 1.341845147189157, 2: 0.007321114355616614, 3: 0.007305302789148918}, 5: {0: 0.9998367848131946, 1: 0.009814212142624035, 2: 0.009761461547859325, 3: 0.009715602263061835}, 6: {0: 0.009890773345421615, 1: 0.42654948276016863, 2: 1.0124372021863575, 3: 0.006878475896675452}, 7: {0: 0.009718486702657828, 1: 0.00971860503040927, 2: 0.25680997365529284, 3: 1.0249670941874818}, 8: {0: 0.9975066063143551, 1: 0.004310521298882478, 2: 0.004320674695405076, 3: 0.0049326455998453134}, 9: {0: 0.007682987834206322, 1: 0.0076950268610715584, 2: 0.0077070321667204

0.0